In [8]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import random
from skimage.feature import hog # Import the hog functio

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def augment_image(img):
    augmented_images = []

    # Flip horizontal
    flipped = cv2.flip(img, 1)
    augmented_images.append(flipped)

    # Rotation
    for angle in [15, -15]:
        (h, w) = img.shape[:2]
        M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
        rotated = cv2.warpAffine(img, M, (w, h))
        augmented_images.append(rotated)

    # Brightness adjustment
    bright = cv2.convertScaleAbs(img, alpha=1.2, beta=30)
    augmented_images.append(bright)

    # Add Gaussian noise
    noise = np.random.normal(0, 10, img.shape).astype(np.uint8)
    noisy = cv2.add(img, noise)
    augmented_images.append(noisy)

    return augmented_images

In [6]:
def load_images(folder):
    features, labels = [], []
    for label in ['GOOD', 'BAD']:
        p = os.path.join(folder, label)
        for fn in os.listdir(p):
            filepath = os.path.join(p, fn)
            img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, (128, 128))

            # Ekstrak HOG dari gambar asli
            hog_vec = hog(img, pixels_per_cell=(16, 16), cells_per_block=(2, 2),
                          orientations=9, block_norm='L2-Hys')
            features.append(hog_vec)
            labels.append(label)

            # Augmentasi dan ekstrak HOG
            augmented_imgs = augment_image(img)
            for aug in augmented_imgs:
                aug = cv2.resize(aug, (128, 128))
                hog_aug = hog(aug, pixels_per_cell=(16, 16), cells_per_block=(2, 2),
                              orientations=9, block_norm='L2-Hys')
                features.append(hog_aug)
                labels.append(label)

    return np.array(features), np.array(labels)

In [9]:
X, y = load_images('/content/drive/MyDrive/CARROT')
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)


In [10]:
svm = SVC(kernel='linear', C=1.0, probability=True)
svm.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [11]:
# Evaluasi
y_pred = svm.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[660   0]
 [  0 633]]
              precision    recall  f1-score   support

         BAD       1.00      1.00      1.00       660
        GOOD       1.00      1.00      1.00       633

    accuracy                           1.00      1293
   macro avg       1.00      1.00      1.00      1293
weighted avg       1.00      1.00      1.00      1293



In [12]:
# Simpan model
with open('model_carrot.pkl', 'wb') as f:
    pickle.dump(svm, f)

print("Model saved as model_carrot.pkl")

Model saved as model_carrot.pkl
